# UBS - Interactive Exploratory Data Analysis

### The following example aims to show the ability of working with:
* Databases, Queries, Data Manipulation using:
  * SQL
  * Python:
      - Pandas
      - Interactive Visualizations

>The aim of this brief exercise is to shed light on the possibilities of crunching large amounts of data from multiple sources seamlessly.
>
>>The notebook provides the **option to only view results** for those not interested in the looking at the code, data traceability and so on, but also the **option to view** the applied logic, syntax and methodology used for the analysis with a simple click.

<h3>Import necessary libraries</h3>

The following lines provide a description of necessary libraries needed to create an exploratory data-model

In [7]:
# Import required libraries
import pandas as pd
import requests
import io
import sqlite3
%load_ext sql
import sqlalchemy as sqla
from sqlalchemy import create_engine
from sqlite3 import Error
import qgrid
    
# Download csv files from GitHub repository
# Note: csv files can be imported from your local machine or network shared folders

url = 'https://raw.githubusercontent.com/LuisFRoch/UBS_Sample/master/Credit_risk.csv'
d0 = requests.get(url).content

url1 = 'https://raw.githubusercontent.com/LuisFRoch/UBS_Sample/master/eba_List_RepInst.csv'
d1 = requests.get(url1).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(d0.decode('utf-8')))
df1 = pd.read_csv(io.StringIO(d1.decode('utf-8')))

<h2>SQL Database and Query Analysis<.h2>

**We create an engine in order to be able to create a virtual database to hold our data :**

In [8]:
engine = create_engine('sqlite:///Creditrisk.db', echo=False)
engine

Engine(sqlite:///Creditrisk.db)

**Once the virtual database is created, we create a connection in order to be able to insert data.**

In [9]:
con = sqlite3.connect('Creditrisk.db')
c = con.cursor()
print('Connected to: ', con)

Connected to:  <sqlite3.Connection object at 0x7f879bc72570>


>**We use the following script to be able to use SQL and Python commands in the same environment**

In [10]:
%sql sqlite:///Creditrisk.db

'Connected: @Creditrisk.db'

**Create SQL tables to hold our downloaded data**

In [11]:
c.execute('''CREATE TABLE IF NOT EXISTS eba_List (LEI_code TEXT, Name TEXT, Country TEXT)''')

c.execute('''CREATE TABLE IF NOT EXISTS Portfolio_list (LEI_code TEXT, 
                                                        NSA TEXT,
                                                        Periods DATE,
                                                        Item REAL,
                                                        Portfolio INTEGER,
                                                        Country INTEGER,
                                                        Country_rank INTEGER,
                                                        Exposure REAL,
                                                        Status INTEGER,
                                                        Perf_Status INTEGER,
                                                        AMOUNT REAL
                                                        )''')
con.commit()

**Load data held in memory and transform SQL table format**

In [12]:
df1 = pd.read_csv(io.StringIO(d1.decode('utf-8')))
df1.to_sql('EBA_List', con, if_exists='append', index = False, chunksize = 1000)
df = pd.read_csv(io.StringIO(d0.decode('utf-8')))
df.to_sql('Portfolio_list', con=engine, if_exists='append', index = False, chunksize = 10000)

**The following lines query the created database to review loaded tables**

In [32]:
%%sql
select *
from EBA_List
Limit 10

 * sqlite:///Creditrisk.db
Done.


LEI_code,Name,Country
PQOH26KWDF7CG10L6792,Erste Group Bank AG,Austria
5299004SNO5GECIBWJ18,Promontoria Sacher Holding N.V.,Austria
529900XSTAE561178282,Raiffeisenbankengruppe OÖ Verbund eGen,Austria
529900SXEWPJ1MRRX537,Raiffeisen-Holding Niederösterreich-Wien Registrierte Genossenschaft Mit Beschränkter Haftung,Austria
EVOYOND2GGP3UHGGE885,Raiffeisen Zentralbank Österreich Aktiengesellschaft,Austria
529900IZ8TASAYR3A694,Sberbank Europe AG,Austria
D1HEB8VEU6D9M8ZUXG17,UniCredit Bank Austria AG,Austria
529900D4CD6DIB3CI904,Volksbanken Wien-Baden AG,Austria
15DYKVGPQCMYBH2DZ583,VTB Bank AG,Austria
LSGM84136ACA92XCN876,AXA Bank Europe SA,Belgium


In [15]:
%%sql
select *
from Portfolio_list
Limit 10

 * sqlite:///Creditrisk.db
Done.


LEI_code,NSA,Periods,Item,Portfolio,Country,Country_rank,Exposure,Status,Perf_Status,AMOUNT
PQOH26KWDF7CG10L6792,AT,201412,150501.0,1,0,0,0.0,0,0,71177.04616499999
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,103.0,0,0,34204.300706
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,104.0,0,0,4944.8083008
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,105.0,0,0,3364.7812408
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,106.0,0,0,550.03152909
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,107.0,0,0,440.30131481
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,203.0,0,0,1897.6831549000003
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,303.0,0,0,10622.100938
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,404.0,0,0,4893.4456837
PQOH26KWDF7CG10L6792,AT,201412,150502.0,1,0,0,501.0,0,0,2840.9543862


In [33]:
%%sql
select count(*)
from EBA_list

 * sqlite:///Creditrisk.db
Done.


count(*)
380


In [34]:
%%sql
select count(*)
from Portfolio_list

 * sqlite:///Creditrisk.db
Done.


count(*)
251666


**Create a "View" or virtual table joining EBA_ List and Portfolio_List tables**

In [18]:
%%sql
create view v_master as
    select pl.LEI_code,
            ebalst.Name as Entity_Name,
            ebalst.Country,
            pl.NSA,
            pl.Periods as Maturity,
            pl.Portfolio,
            pl.Exposure,
            pl.Perf_Status, 
            pl.AMOUNT
from Portfolio_list as pl
left join EBA_List as ebalst
on pl.LEI_code = ebalst.LEI_code

 * sqlite:///Creditrisk.db
(sqlite3.OperationalError) table v_master already exists
[SQL: create view v_master as
    select pl.LEI_code,
            ebalst.Name as Entity_Name,
            ebalst.Country,
            pl.NSA,
            pl.Periods as Maturity,
            pl.Portfolio,
            pl.Exposure,
            pl.Perf_Status, 
            pl.AMOUNT
from Portfolio_list as pl
left join EBA_List as ebalst
on pl.LEI_code = ebalst.LEI_code]
(Background on this error at: http://sqlalche.me/e/e3q8)


**Query created master table**

In [19]:
%%sql
select *
from v_master
Limit 10

 * sqlite:///Creditrisk.db
Done.


LEI_code,NSA,Maturity,Portfolio,Exposure,Perf_Status,AMOUNT,Country,Entity_Name
PQOH26KWDF7CG10L6792,AT,201412,1,0.0,0,71177.04616499999,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,0.0,0,71177.04616499999,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,103.0,0,34204.300706,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,103.0,0,34204.300706,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,104.0,0,4944.8083008,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,104.0,0,4944.8083008,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,105.0,0,3364.7812408,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,105.0,0,3364.7812408,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,106.0,0,550.03152909,Austria,Erste Group Bank AG
PQOH26KWDF7CG10L6792,AT,201412,1,106.0,0,550.03152909,Austria,Erste Group Bank AG


**Simple sum of amount invested grouping by Maturity, Country and Enitty**

In [21]:
%%sql
Select  Maturity,
        substr(Maturity, 1, 4) as Yr,
        substr(Maturity, 6, 2) as Month,
        Country,
        NSA,
        Entity_Name,
        Portfolio,
        Exposure,
        sum(AMOUNT) as Total_Invested
From v_master
Where Exposure > 500
Group by Yr, Month, Country, Entity_Name
Order by Yr, Total_Invested
Limit 4

 * sqlite:///Creditrisk.db
Done.


Maturity,Yr,Month,Country,NSA,Entity_Name,Portfolio,Exposure,Total_Invested
201412,2014,2,Denmark,DK,Sydbank A/S,1,501.0,3353.8069638653997
201412,2014,2,Latvia,LV,ABLV Bank,1,501.0,3862.1456887800546
201412,2014,2,Germany,DE,Deutsche Apotheker- und Ärztebank eG,1,501.0,4203.3306609400015
201412,2014,2,Ireland,IE,Permanent TSB Group Holdings Plc,1,501.0,9080.1195430132


<h2>Transform SQL data into Pythonic language for further analysis</h2>

### Import SQL dataset into a dataframe

In [23]:
df = pd.read_sql('select * from v_master', con)
df.head()

,LEI_code,NSA,Maturity,Portfolio,Exposure,Perf_Status,AMOUNT,Country,Entity_Name
0,PQOH26KWDF7CG10L6792,AT,201412,1,0.0,0,71177.046165,Austria,Erste Group Bank AG
1,PQOH26KWDF7CG10L6792,AT,201412,1,0.0,0,71177.046165,Austria,Erste Group Bank AG
2,PQOH26KWDF7CG10L6792,AT,201412,1,103.0,0,34204.300706,Austria,Erste Group Bank AG
3,PQOH26KWDF7CG10L6792,AT,201412,1,103.0,0,34204.300706,Austria,Erste Group Bank AG
4,PQOH26KWDF7CG10L6792,AT,201412,1,104.0,0,4944.808301,Austria,Erste Group Bank AG


###  View dataset number of rows, columns and missing values

In [25]:
# Dataset number of rows and columns

print ('View dataset number of rows and columns: ', df.shape)

View dataset number of rows and columns:  (476042, 9)
